In [1]:
import requests, json
from tqdm import tqdm
from datetime import datetime
import pandas as pd
import urllib

api = '2A955004026F4AEAAFA21FBCCA4A65FB'

In [2]:
# paste asins here
asins = [x for x in """

B000VBY072
B07WPFC22Y
B00063GP5K
B00ADVCRZM
B005H90IX8
B008OHY1RE
B00BAWOAZI
B000VBUNDC
B01BEAVON6
B01ETNR45W
B01ETNR5EM
B07FZDR43M
B00KBB9D42
B00CDUAEU6
B076BSXF6S
B0051UVJNG
B076BV5S3W
B01EVNQHZS
B07FYWVSHX
B07FZCCTXT
B07M5CFFFG
B06ZZC6M16


""".split('\n') if x != '' and len(x) > 4]
print(len(asins))
print(len(set(asins)))

22
22


In [3]:
# make collection
def make_coll():
    body = {
      "name": datetime.strftime(datetime.now(), '%Y-%m-%d %H:%M'),
      "enabled": True,
      "schedule_type": "manual"
    }
    api_result = requests.post(
        'https://api.rainforestapi.com/collections?api_key={}'.format(api), 
        json=body
    )
    api_response = api_result.json()
    col = api_response['collection']['id']
    return col

# make offers payload
def make_payload(type_, type_key, list_):
    requests_ = []
    for asin in list_:
        request = {
            'type': type_,
            'amazon_domain': 'amazon.com',
            type_key: asin,
            'custom_id': '123'
        }
        requests_.append(request)
    body = {
        'requests': requests_
    }
    return body

# add to collection
def add_to_coll(col, body):
    params = {
      'api_key': api
    }
    result = requests.put(
        'https://api.rainforestapi.com/collections/{}?api_key={}'.format(col, api), 
        json=body
    )
    r = result.json()
    return r

# start col
def start_coll(col):
    params = {
      'api_key': api
    }
    result = requests.get('https://api.rainforestapi.com/collections/{}/start'.format(col), params)
    r = result.json()
    return r

# check col
def check_coll(col):
    params = {
      'api_key': api
    }
    result = requests.get(
        'https://api.rainforestapi.com/collections/{}'.format(col), 
        params
    )
    response = result.json()
    return response

# get download link
def get_dl(col):
    params = {
      'api_key': api
    }
    api_result = requests.get(
        'https://api.rainforestapi.com/collections/{}/results/1'.format(col), 
        params
    )
    r = api_result.json()
    url = r['result']['download_links']['pages'][0]
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    return data

In [4]:
col = make_coll()

In [5]:
body = make_payload('product', 'asin', list(set(asins)))

In [6]:
add_to_coll(col, body)

{'request_info': {'success': True},
 'collection': {'id': '42FCBB8D',
  'created_at': '2021-05-25T02:58:11.059Z',
  'name': '2021-05-25 11:58',
  'schedule_type': 'manual',
  'enabled': True,
  'status': 'idle',
  'requests_total_count': 22,
  'requests_page_count': 1,
  'credits_required': 22,
  'results_count': 0,
  'priority': 'normal',
  'destination_ids': [],
  'requests_type': 'product',
  'requests_per_type_count': {'product': 22},
  'requests_type_locked': False}}

In [7]:
start_coll(col)

{'request_info': {'success': True},
 'collection': {'id': '42FCBB8D',
  'created_at': '2021-05-25T02:58:11.059Z',
  'name': '2021-05-25 11:58',
  'schedule_type': 'manual',
  'enabled': True,
  'status': 'queued',
  'requests_total_count': 22,
  'requests_page_count': 1,
  'credits_required': 22,
  'results_count': 0,
  'priority': 'normal',
  'destination_ids': [],
  'requests_type': 'product',
  'requests_per_type_count': {'product': 22},
  'requests_type_locked': False}}

In [9]:
check_coll(col)

{'request_info': {'success': True},
 'collection': {'id': '42FCBB8D',
  'created_at': '2021-05-25T02:58:11.059Z',
  'name': '2021-05-25 11:58',
  'schedule_type': 'manual',
  'enabled': True,
  'status': 'idle',
  'requests_total_count': 22,
  'requests_page_count': 1,
  'credits_required': 22,
  'results_count': 1,
  'priority': 'normal',
  'destination_ids': [],
  'requests_type': 'product',
  'requests_per_type_count': {'product': 22},
  'requests_type_locked': False}}

In [10]:
data = get_dl(col)

In [11]:
data[0]['result']['product']['feature_bullets']

KeyError: 'feature_bullets'

In [12]:
r = []
for x in data:
    row = []
    try:
        row.append(x['request']['asin'])
        for y in x['result']['product']['feature_bullets']:
            row.append(y)
        # row.append(x['result']['product']['feature_bullets'])
    except:
        pass
    r.append(row)

In [13]:
pd.DataFrame(r)

,0,1,2,3,4,5
0,B01ETNR5EM,None,None,None,None,None
1,B00ADVCRZM,Totem Strawberries,None,None,None,None
2,B005H90IX8,None,None,None,None,None
3,B076BSXF6S,Exquisite flavor,"We are proud to offer a simple, delicious, and...",Our Fruit Pates distill the pure essence of ou...,None,None
4,B07FYWVSHX,Specialty,None,None,None,None
5,B000VBUNDC,Our NW Red Raspberry jam will soon earn a perm...,Packed with fresh Willamette Raspberries and n...,Our jams contain 1/3 less sugar than tradition...,None,None
6,B01EVNQHZS,None,None,None,None,None
7,B07M5CFFFG,"Raw, unfiltered, unpasteurized, natural, pure ...","Superfood constituents: bee pollen, honey comb...",Our honey is raw and unfiltered to preserve th...,None,None
8,B000VBY072,Perfect on pancakes and toast,None,None,None,None
9,B07FZCCTXT,Honey is raw and unfiltered,None,None,None,None


In [14]:
pd.DataFrame(r).to_csv('bullets.csv')